In [143]:
import keras
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import pickle as p
import matplotlib.pyplot as plt
from scipy import *
import os
import h5py
from keras import backend as K
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD, adam
from keras.models import Sequential
from keras.losses import categorical_crossentropy
from sklearn.utils import shuffle
from keras.utils import np_utils
import matplotlib.pyplot as plt
from skimage.feature import hog
from skimage import data, exposure
import pickle 
from skimage.transform import rescale, resize
import scipy.misc
from astropy.io import ascii

import dask.dataframe as dd

In [144]:
# Setting directory paths 
root_dir = os.getcwd()

# Loading data
data = h5py.File(r'data.h5', 'r')

images_raw = np.asarray(data['images'])
labels = data['labels']

In [145]:
FIRST = ascii.read("asu.tsv").to_pandas()

In [146]:
FIRST

,Name,RAJ2000,DEJ2000,True,Pred,Prob,CoNFIG,B12,NVSS,FIRST
0,NaN,h:m:s,d:m:s,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-------------------,------------,------------,----,-------,---------,---,---,---,---
2,1426+0093,14 26 49.840,+00 55 59.90,FRII,FRII,99.99995,1,0,1,0
3,3C 194,08 10 03.670,+42 28 04.00,FRII,FRII,99.99995,2,0,1,0
4,3C 208,08 53 08.830,+13 52 55.30,FRII,FRII,99.99735,2,0,1,1
5,3C 228,09 50 10.770,+14 19 57.30,FRII,FRII,99.99990,2,0,1,0
6,3C 240,10 17 49.770,+27 32 07.70,FRII,FRII,99.99785,1,0,1,0
7,3C 243,10 26 31.960,+06 27 32.70,FRII,FRII,100.00000,1,0,1,0
8,3C 244.1,10 33 33.870,+58 14 37.90,FRII,FRII,99.99980,2,0,1,0
9,3C 251,11 08 37.600,+38 58 42.10,FRII,FRII,99.99810,1,0,0,0


In [197]:
nb_fri = FIRST[FIRST["True"] == 'FRI'].shape[0]
nb_frii = FIRST[FIRST["True"] == 'FRII'].shape[0]

In [198]:
frii_names_n = np.asarray(data['frii_data']['SDSS']).tolist()
fri_names_n = np.asarray(data['fri_data']['name']).tolist()

frii_data = data['frii_data']
fri_data = data['fri_data']

In [199]:
frii_c = np.hstack((np.asarray(list(frii_data['_RA'])).reshape(-1, 1), np.asarray(list(frii_data['_DE'])).reshape(-1, 1)))
fri_c = np.hstack((np.asarray(list(fri_data['ra'])).astype(str).reshape((-1, 1)), np.asarray(list(fri_data['dec'])).astype(str).reshape((-1, 1))))

In [214]:
fri_c

array([['00 39 30.52', '-10 32 18.6'],
       ['01 12 55.11', '-09 50 40.6'],
       ['00 41 48.22', '-09 17 03.1'],
       ['00 43 00.63', '-09 13 46.3'],
       ['22 31 43.19', '-08 24 31.7'],
       ['01 33 27.25', '-08 24 16.4'],
       ['21 42 39.29', '-08 04 23.8'],
       ['02 52 27.52', '-07 56 05.4'],
       ['21 20 05.00', '-07 53 50.1'],
       ['12 54 34.93', '-02 34 12.4'],
       ['13 10 53.44', '-02 28 41.5'],
       ['12 09 43.62', '-02 04 59.6'],
       ['12 00 21.93', '-02 01 52.7'],
       ['00 29 00.98', '-01 13 41.7'],
       ['01 34 12.78', '-01 07 29.4'],
       ['23 47 02.42', '-01 03 00.9'],
       ['01 35 03.43', '-00 54 27.6'],
       ['13 02 03.58', '-00 50 12.3'],
       ['00 45 30.46', '-00 47 46.9'],
       ['10 49 21.13', '-00 40 05.0'],
       ['01 52 53.79', '-00 10 05.5'],
       ['01 40 29.59', '+00 18 25.8'],
       ['10 19 37.94', '+00 19 55.7'],
       ['14 26 16.34', '+00 50 15.3'],
       ['02 54 37.99', '+00 56 21.9'],
       ['14 36 38.56', '+

In [212]:
# Fetching Indices
ind_first = np.arange(2, int(FIRST.shape[0]))
ind_fri = np.arange(0, fri_c.shape[0])
ind_frii = np.arange(0, frii_c.shape[0])

In [218]:
from astropy.coordinates import SkyCoord

# Coverting coordinates of wildly varying formats to SkyCoords
coord_first = SkyCoord(ra=FIRST.iloc[ind_first]['RAJ2000'], dec=FIRST.iloc[ind_first]['DEJ2000'], unit=('hourangle', 'deg'))
coord_fri = SkyCoord(ra=fri_c[:, 0][ind_fri], dec=fri_c[:, 1][ind_fri], unit=('hourangle', 'deg'))
coord_frii = SkyCoord(ra=frii_c[:, 0][ind_frii], dec=frii_c[:, 1][ind_frii], unit=('hourangle', 'deg'))

In [223]:
# Pulling out objects from FR1 and FR2 that are in asu.tsv
fri_in_asu = []
frii_in_asu = []
asu_dud = []


for index, coord in enumerate(coord_first):
    not_found = 0
    
    for i, fri_coord in enumerate(coord_fri):
        if coord == fri_coord:
            fri_in_asu.append(i)
            break
        
        elif i == (coord_fri.shape[0]-1):
            not_found += 1
        
    
            
    for i, frii_coord in enumerate(coord_fri):
        if coord == frii_coord:
            frii_in_asu.append(i)
            break
        
        elif i == (coord_frii.shape[0]-1):
            not_found += 1   
    
    if not_found == 2:
        asu_dud.append(index)
    elif not_found < 2:
        continue
    else:
        print ('error')
        print (not_found)

In [226]:
coord_first.shape
np.asarray(asu_dud).shape

(187,)

In [179]:
print(list(frii_data.keys()))
print(list(fri_data.keys()))

['Class', 'Cr', 'Dn4000', 'F_OIII_', 'S1.4GHz', 'SDSS', 'SimbadName', 'Sloan', '_DE', '_RA', 'logL_OIII_', 'logMBH', 'lognuLr', 'rmag', 'sigma', 'z']
['Unnamed: 0', 'Unnamed: 17', 'bii', 'concentration_index', 'dec', 'dn_4000_index', 'galaxy_sample', 'lii', 'log_1p4_ghz_luminosity', 'log_black_hole_mass', 'log_o_iii_luminosity', 'name', 'nvss_flux_1p4_ghz', 'o_iii_flux', 'ra', 'redshift', 'rmag', 'stellar_dispersion']


In [175]:
def to_str(data, fr):
    nb = len(data)
    str_data = []
    
    if fr == 1:
        for i in range(nb):
            str_data.append(str(data[i])[7:-1])
    elif fr == 2:
        for i in range(nb):
            str_data.append(str(data[i])[2:-1])
        
        
    return str_data

In [176]:
fri_names = to_str(fri, 1)
frii_names = to_str(frii, 2)

NameError: name 'fri' is not defined

In [ ]:
fri_components = np.array([])
frii_components = np.array([])

In [10]:
fri_names

['J003930.52-103218.6',
 'J011255.11-095040.6',
 'J004148.22-091703.1',
 'J004300.63-091346.3',
 'J223143.19-082431.7',
 'J013327.25-082416.4',
 'J214239.29-080423.8',
 'J025227.52-075605.4',
 'J212005.00-075350.1',
 'J125434.93-023412.4',
 'J131053.44-022841.5',
 'J120943.62-020459.6',
 'J120021.93-020152.7',
 'J002900.98-011341.7',
 'J013412.78-010729.4',
 'J234702.42-010300.9',
 'J013503.43-005427.6',
 'J130203.58-005012.3',
 'J004530.46-004746.9',
 'J104921.13-004005.0',
 'J015253.79-001005.5',
 'J014029.59+001825.8',
 'J101937.94+001955.7',
 'J142616.34+005015.3',
 'J025437.99+005621.9',
 'J143638.56+011058.8',
 'J154155.16+012517.4',
 'J153932.09+013710.5',
 'J122156.16+020450.8',
 'J121640.12+034231.5',
 'J120425.29+034510.6',
 'J095527.76+034516.8',
 'J093058.74+034827.7',
 'J081849.74+040631.5',
 'J152122.54+042030.1',
 'J132017.54+043037.4',
 'J143257.81+043715.1',
 'J150450.51+044054.8',
 'J120522.29+050941.4',
 'J134529.50+054952.9',
 'J140916.74+060139.4',
 'J121110.99+060

In [11]:
frii_names

['J001247.57+004715.8',
 'J002107.62-005531.4',
 'J004326.80-105421.8',
 'J004404.66+010152.9',
 'J011830.65-104356.4',
 'J024558.54-064900.6',
 'J075221.83+333348.9',
 'J075529.95+520450.6',
 'J075628.78+501716.3',
 'J080107.19+175849.7',
 'J080158.09+175152.6',
 'J080535.00+240950.3',
 'J081343.61+525738.2',
 'J081512.33+384045.4',
 'J081734.46+445850.8',
 'J082247.63+041750.0',
 'J083109.54+414738.1',
 'J083112.15+434158.4',
 'J084529.04+421952.9',
 'J084759.04+314708.3',
 'J090150.32+555527.4',
 'J091153.61+372413.3',
 'J091225.08+534139.1',
 'J091445.53+413714.2',
 'J091948.45+575055.9',
 'J092611.26+170357.9',
 'J093215.91+180419.9',
 'J093641.99+111350.9',
 'J094124.02+394441.8',
 'J094201.76+084736.8',
 'J094703.01+231614.2',
 'J095640.77-000124.0',
 'J101558.24+404647.2',
 'J101954.48+393022.8',
 'J102150.38+080833.8',
 'J102156.67+144331.4',
 'J102237.40+383444.9',
 'J103128.22+084324.1',
 'J103443.26+053319.8',
 'J103602.94+525936.1',
 'J104502.74+471759.1',
 'J104632.21+543

In [100]:
# balance classes
def bal_classes(images, labels):
    class1 = []
    class2 = []
    
    for i in range(labels.shape[0]):
        if labels[i] == False:
            class1.append(i)
    
    for i in range(labels.shape[0]):
        if labels[i] == True:
            class2.append(i)
    
    if len(class2) > len(class1):
        class2 = np.asarray(class2[:len(class1)])
    
    elif len(class2) < len(class1):
        class1 = np.asarray(class1[:len(class2)])
        
    images1 = images[class1, :] 
    images2 = images[class2, :]
    labels1 = labels[class1,]
    labels2 = labels[class2,]
    
    print (labels2.shape)
    
    images_st = np.vstack((images1, images2))
    labels_st =  np.concatenate((labels1, labels2), axis=0)              
        
    images_s, labels_s = shuffle(np.asarray(images_st), np.asarray(labels_st), random_state = 0)    
        
    return images_s, labels_s

In [5]:
images_p, labels_p = bal_classes(np.asarray(images_raw), np.asarray(labels))

(123,)


In [6]:
# Defining rescaling function
def rescale_imgs(images):
    count =  0
    for i in range(0 ,images.shape[0]):
        img_scaled = expand_dims(resize(images[i, :, :], (384, 384)), axis = 0)
        if count == 0:
            imgs = img_scaled
        else:
            imgs = np.vstack((imgs, img_scaled))
        count += 1
                             
    return imgs

In [7]:
images_shrunk = rescale_imgs(images_p)

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [8]:
print (images_shrunk.shape)

(246, 384, 384)


In [9]:
def format_data(images, labels, ratio = 0.5):
    split = round(ratio*images.shape[0])
    
    train_x = images[ :split, :]
    test_x = images[split:, :]
    train_y = labels[ :split]
    test_y = labels[split:]
    
    return train_x, test_x, train_y, test_y


train_x, test_x, train_y, test_y = format_data(images_shrunk, labels_p)

In [10]:
train_x.shape

(123, 384, 384)

In [11]:
#imgs = np.reshape(images_shrunk, (-1, images_shrunk.shape[1], images_shrunk.shape[2]))

#plt.figure(figsize=(12, 12))
#for n in range(0, 16):
    #plt.subplot(4, 4, 1 + n)
    #plt.imshow(stretch(imgs[-n-1,:,:]), cmap='viridis', shape=(135, 135))
    #plt.axis('off')
    #plt.title(labels[n])

In [12]:
#stretch = vis.AsinhStretch(1e-1) + vis.MinMaxInterval()

#images_s = np.zeros((images_shrunk.shape[0], images_shrunk.shape[1], images_shrunk.shape[2]))

#for i, images in enumerate(images_shrunk):
    #images_s[i, :, :] = stretch(images_shrunk[i, :, :])

In [13]:
def data_gen(images, labels):

    batch_size = 20
    samples = np.asarray(images).shape[0]

    datagen = ImageDataGenerator(
            rotation_range=180,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip =True,
            fill_mode = "nearest")

    x = images.reshape((-1, images.shape[1],images.shape[2], 1))

    # the .flow() command below generates batches of randomly transformed images
    # and saves the results to the `preview/` directory
    gen = datagen.flow(x, np.asarray(labels), batch_size=batch_size)

    data_mult = 50

    pro_images = np.zeros((images.shape[0]*data_mult, images.shape[1], images.shape[2], 1))
    pro_labels = np.zeros((labels.shape[0]*data_mult))
    
    for e in range(1, data_mult+1):
        batch = 1
        b = batch_size
        b_start = samples*(e-1)

        for X_batch, Y_batch in gen:
            if batch < (samples/b):
                pro_images[b_start+b*(batch-1):b_start+batch*b, :, :, :] = X_batch
                pro_labels[b_start+b*(batch-1):b_start+batch*b] = Y_batch

            else: 
                pro_images[b_start+b*(batch-1):b_start+b*(batch-1) + X_batch.shape[0]%b, :, :, :] = X_batch
                pro_labels[b_start+b*(batch-1):b_start+b*(batch-1) + X_batch.shape[0]%b] = Y_batch
                break


            batch += 1
        print(e)
        
    return pro_images, pro_labels

In [14]:
pro_train_x, pro_train_y = data_gen(train_x, train_y)
pro_test_x, pro_test_y  = data_gen(test_x, test_y)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50


In [15]:
pro_train_x = np.reshape(pro_train_x, (pro_train_x.shape[0], pro_train_x.shape[1], pro_train_x.shape[2]))
pro_test_x = np.reshape(pro_test_x, (pro_test_x.shape[0], pro_test_x.shape[1], pro_test_x.shape[2]))

In [16]:
#imgs = np.reshape(pro_images, (-1, pro_images.shape[1], pro_images.shape[2]))

#for n in range(0, imgs.shape[0]):
   #plt.imshow(imgs[n,:,:], cmap='gray', shape=(89, 89))
   #print (n)
   #print (pro_labels[n])
   #plt.show()

In [ ]:
with open('train_x_VGG.dat', 'wb') as f:
    pickle.dump(pro_train_x, f, protocol=-1)

In [ ]:
with open('test_x_VGG.dat', 'wb') as f:
    pickle.dump(pro_test_x, f, protocol=-1)

In [ ]:
with open('train_y_VGG.dat', 'wb') as f:
    pickle.dump(pro_train_y, f, protocol=-1)    

with open('test_y_VGG.dat', 'wb') as f:
    pickle.dump(pro_test_y, f, protocol=-1)

In [ ]:
print ("Done")

In [ ]:
with open('raw_train_x.dat', 'wb') as f:
    pickle.dump(pro_train_x, f, protocol=-1)
    
with open('raw_test_x.dat', 'wb') as f:
    pickle.dump(pro_test_x, f, protocol=-1)

In [ ]:
# Define function to crop off 'cut' number of pixels on both sides on both axixs
def crop_center(img,cut):
    y = np.shape(img)[1]
    x = np.shape(img)[2]
    if x != y:
        print ("The image is not a perfect sqaure. This is bad. Fix it")
        
    start = cut
    end = x-cut
    return img[:, start:end,start:end]